## Esercizio

Estendere il linguaggio MiniCaml e l’implementazione dell’interprete con il nuovo tipo di dato “lista di interi” con le seguenti operazioni (già in sintassi astratta):
* `Empty` descrive una lista di interi vuota
* `Cons(e1,e2)`, dove e1 descrive un intero e e2 una lista, produce la lista ottenuta aggiungendo l’intero
descritto da e1 in testa alla lista descritta da e2
* `Length(e)`, dove e descrive una lista, calcola la lunghezza di tale lista
* `SumAll(e)`, dove e descrive una lista, calcola la somma degli elementi di tale lista

Dividiamo tutti gli "oggetti" di interesse in tre categorie:
- Espressioni (o sintassi) MiniCaml (tipo `exp`)
- Valori MiniCaml (tipo `evT`)
- Valori OCaml

Creare un nuovo _valore_ MiniCaml (lista di interi).

Creare nuove _espressioni_ MiniCaml (Empty, Cons, Length, SumAll).

Utilizzare i _valori OCaml_ `int list` per rappresentare il nuovo valore MiniCaml (lista di interi).

Estendiamo MiniCaml anche con le Bifun

Creare nuove _espressioni_ MiniCaml (`Bifun`, `BiApply`)

Creare nuovi _valori_ MiniCaml (`BiClosure`)

### Espressioni

In [1]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound | TListInt | TBiClosure

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* Liste di Interi *)
    | Empty
    | Cons of exp * exp
    | Length of exp
    | SumAll of exp
    (* Bifun *)
    | Bifun of ide * ide * exp
    | BiApply of exp * exp * exp
;;

type ide = string


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TUnBound
  | TListInt
  | TBiClosure


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | Empty
  | Cons of exp * exp
  | Length of exp
  | SumAll of exp
  | Bifun of ide * ide * exp
  | BiApply of exp * exp * exp


### Tipi esprimibili

In [2]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int  
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env
    | RecClosure of ide * ide * exp * evT env
    | UnBound
    (* TListInt *)
    | ListInt of int list
    (* TBiclosure *)
    | Biclosure of ide * ide * exp * evT env

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | UnBound
  | ListInt of int list
  | Biclosure of ide * ide * exp * evT env


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


### Typechecking

In [3]:
(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound
    | ListInt(_) -> TListInt

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )
    | TListInt -> (match y with
                 | ListInt(_) -> true
                 | _ -> false
                 )
    

File "[3]", line 3, characters 4-218:
 3 | ....match x with
 4 |     | Int(n) -> TInt
 5 |     | Bool(b) -> TBool
 6 |     | String(s) -> TString
 7 |     | Closure(i,e,en) -> TClosure
 8 |     | RecClosure(i,j,e,en) -> TRecClosure
 9 |     | UnBound -> TUnBound
10 |     | ListInt(_) -> TListInt
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Biclosure (_, _, _, _)


val getType : evT -> tname = <fun>


File "[3]", line 14, characters 4-857:
14 | ....match x with
15 |     | TInt -> (match y with 
16 |                | Int(u) -> true
17 |                | _ -> false
18 |                )
...
39 |     | TListInt -> (match y with
40 |                  | ListInt(_) -> true
41 |                  | _ -> false
42 |                  )
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
TBiClosure


val typecheck : tname * evT -> bool = <fun>


### Interprete

In [4]:
(* Errori a runtime *)
exception RuntimeError of string

(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

exception RuntimeError of string


val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


...

val less_than : evT * evT -> evT = <fun>


val greater_than : evT * evT -> evT = <fun>


val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


In [5]:
let sum (l: int list) : int =
  List.fold_left (+) 0 l;;

let rec sum (l: int list) : int =
    match l with
    | [] -> 0
    | x::xs -> x + sum xs;;

val sum : int list -> int = <fun>


val sum : int list -> int = <fun>


In [6]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (* ListInt *)
    | Empty -> ListInt([])
    | Length(e) -> (* e: exp *)
        let (l: evT) = eval e s in
        (match typecheck(TListInt, l), l with
        | true, ListInt(locaml) -> Int (List.length locaml)
        | _ -> raise ( RuntimeError "Wrong type")
        )
    | SumAll(e) ->
        let l = eval e s in
        (match typecheck(TListInt, l), l with
        | true, ListInt(locaml) -> Int (sum locaml)
        | _ -> raise ( RuntimeError "Wrong type")
        )
    | Cons(e1, e2) ->
        let x = eval e1 s in
        let l = eval e2 s in
        (match x, l with
        | Int(a), ListInt(locaml) -> ListInt(a::locaml)
        | _ -> raise ( RuntimeError "Wrong type")
        )
    (* Bifun *)
    | Bifun(x, y, body) -> Biclosure(x, y, body, s)
    | BiApply(efun, ex, ey) ->
        let fclosure = eval efun s in
        (match fclosure with
        | Biclosure(argx, argy, body, fEnv) ->
            let xval = eval ex s in
            let yval = eval ey s in
            let env1 = bind fEnv argx xval in
            let env2 = bind env1 argy yval in
                eval body env2
        | _ -> raise ( RuntimeError "Wrong type")
        )


val eval : exp -> evT env -> evT = <fun>


In [7]:
let test1 = SumAll(Cons(EInt(1),Cons(EInt(2),Empty)));;

eval test1 emptyenv;;

val test1 : exp = SumAll (Cons (EInt 1, Cons (EInt 2, Empty)))


- : evT = Int 3


In [8]:
let test2 = Let(
    "somma",
    Bifun("x", "y", Sum(Den "x", Den "y")),
    BiApply (Den "somma", EInt 3, EInt 2)
);;

eval test2 emptyenv;;

val test2 : exp =
  Let ("somma", Bifun ("x", "y", Sum (Den "x", Den "y")),
   BiApply (Den "somma", EInt 3, EInt 2))


- : evT = Int 5
